# Objectives of this Lab
- understand the role of material "representations"

## Reminders
- activate your hsi25_ml-ssc_25.0 conda env (kernel)

# This time
- representing complex data (primarily molecules and materials)

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

plt.style.use('../files/plot_style.mplstyle')

In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.core.composition import Composition

from matminer.datasets import load_dataset
from matminer.featurizers.composition.composite import ElementProperty
from matminer.featurizers.composition.element import ElementFraction
from matminer.featurizers.structure.rdf import RadialDistributionFunction

# Representing crystal structures
- let's take a look at the "FLLA" dataset --> F. Faber, A. Lindmaa, O.A. von Lilienfeld, R. Armiento, “Crystal structure representations for machine learning models of formation energies”, Int. J. Quantum Chem. 115 (2015) 1094–1101. doi:10.1002/qua.24917
- ~4000 inorganic materials
- ground-truth is plane wave density functional theory
- goal is for our ML model to be a surrogate for DFT by giving us a fast estimate of the formation energy (used as input to thermodynamic models)

In [ ]:
df = load_dataset('flla')
del df['formula']

def get_formula(row):
    s = row['structure']
    formula = s.formula
    reduced_formula = Composition(formula).reduced_formula
    return reduced_formula

df['formula'] = df.apply(get_formula, axis=1)

df.head()

# First, we'll get a fingerprint that just captures the chemical composition

In [ ]:
def get_elfrac_composition_vector(row):
    return ElementFraction().featurize(Composition(row['formula']))

df['elfrac_vector'] = df.apply(get_elfrac_composition_vector, axis=1)

df.head()

In [ ]:
LiCl_elfrac = df.elfrac_vector.get(df.formula == 'LiCl').values

In [ ]:
LiCl_elfrac

# Now, we'll get a fingerprint that captures elemental **properties**
- converting chemical formulas into vectors using things like average electronegativity, average radius, etc.

In [ ]:
elprop = ElementProperty.from_preset('matminer')
elprop

In [ ]:
our_props = ElementProperty(data_source='pymatgen', features=['X', 'mendeleev_no', 'atomic_radius', 'melting_point'], stats=['mean', 'std_dev'])
our_props

## Let's look at a random formula in our dataset

In [ ]:
some_formula = df.formula.values[21]
print(some_formula)

## Now convert that formula to our "element property" vector

In [ ]:
our_props.featurize(Composition(some_formula))

## What do those values correspond with?

In [ ]:
def get_features_and_values(formula):
    feature_values = our_props.featurize(Composition(formula))
    feature_labels = our_props.feature_labels()
    labels_to_values = dict(zip(feature_labels, feature_values))
    labels_to_values = {k.strip('PymatgenData').replace(' ', '_')[1:] : v for i, (k, v) in enumerate(labels_to_values.items()) if v}
    return labels_to_values

In [ ]:
get_features_and_values(some_formula)

## Let's apply this to our whole dataset

In [ ]:
def get_matminer_composition_vector(row):    
    elprop = our_props
    formula = row['formula']
    return elprop.featurize(Composition(formula))

df['matminer_vector'] = df.apply(get_matminer_composition_vector, axis=1)

In [ ]:
df.head()

# Let's compare the two fingerprints

In [ ]:
formula = 'NaMnSe2'

elfrac_vector = df['elfrac_vector'].get(df.formula == formula).values[0]
matminer_vector = df['matminer_vector'].get(df.formula == formula).values[0]

print('elfrac_vector has %i features' % len(elfrac_vector))
print('matminer_vector has %i features' % len(matminer_vector))

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(211)
ax1 = plt.bar(list(range(len(elfrac_vector))), elfrac_vector)
ax2 = plt.subplot(212)
ax2 = plt.bar(list(range(len(matminer_vector))), matminer_vector)

# Now we could train models with each. Which one do you think should work better?
- we'll write a quick function to run CV with these models

In [ ]:
def CV(feature):
    # convert our single column containing all features into a feature matrix (each item of the vector becomes a column of the matrix)
    X = np.vstack(df[feature].values)

    # grab our target
    y = df['formation_energy_per_atom'].values

    # split our data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=44)

    # use mean imputation (to fill in NaNs)
    imputer = SimpleImputer(strategy='mean')    

    # initialize a vanilla random forest
    rf = RandomForestRegressor()

    # impute --> fit
    pipe = Pipeline([('impute', imputer), ('rf', rf)])

    # run three-fold CV
    scores = cross_validate(pipe, X_train, y_train, cv=3, return_train_score=True, scoring='neg_root_mean_squared_error')
    print('Training score = %.3f +/- %.4f' % (np.mean(abs(scores['train_score'])), np.std(abs(scores['train_score']))))
    print('Validation score = %.3f +/- %.4f' % (np.mean(abs(scores['test_score'])), np.std(abs(scores['test_score']))))

    # re-train model on full training set (to inspect importances)
    pipe.fit(X_train, y_train)

    # return scores and the model fit to the full training set
    return scores, pipe['rf']

# Now, we'll compare the featurization based on element fraction and based on the element properties

In [ ]:
for feature in ['elfrac_vector', 'matminer_vector']:
    print('~~~~ %s ~~~~' % feature)
    scores, rf = CV(feature)
    print('\n')

### With ~10x fewer features, the element property (matminer vector) does better!

# Let's see which features are most important
- the default built-in `feature_importances_` applies average impurity decrease to rank feature importances

In [ ]:
importances = rf.feature_importances_
importances

In [ ]:
def plot_importances(features_and_their_importances, ylabel='average impurity decrease', n_features_to_plot=10, figsize=(7,5)):
    """
    Args:
        features_and_their_importances (dict):
            {feature (str) : importance (float)}
    Returns:
        matplotlib bar chart of sorted importances
    """
    axis_width = 1.5
    maj_tick_len = 6
    fontsize = 14
    bar_color = 'lightblue'
    align = 'center'
    label = '__nolegend__'
    
    sorted_features = sorted(features_and_their_importances, 
                             key=features_and_their_importances.get, 
                             reverse=True)
    sorted_importances = [features_and_their_importances[f] for f in sorted_features]

    if len(sorted_features) < n_features_to_plot:
        n_features_to_plot = len(sorted_features)

    fig = plt.figure(figsize=figsize)
    ax = plt.bar(range(n_features_to_plot), sorted_importances[:n_features_to_plot],
                 color=bar_color, align=align, label=label)
    ax = plt.xticks(range(n_features_to_plot), sorted_features[:n_features_to_plot], rotation=90)
    ax = plt.xlim([-1, n_features_to_plot])
    ax = plt.ylabel(ylabel, fontsize=fontsize)
    ax = plt.tick_params('both', length=maj_tick_len, width=axis_width, 
                         which='major', right=True, top=True)
    ax = plt.xticks(fontsize=fontsize)
    ax = plt.yticks(fontsize=fontsize)
    ax = plt.tight_layout()
    return ax

In [ ]:
feature_labels = list(get_features_and_values(formula).keys())
features_and_their_importances = dict(zip(feature_labels, importances))
plot_importances(features_and_their_importances)

## Let's compare to the element fraction features

In [ ]:
scores, rf = CV('elfrac_vector')

In [ ]:
importances = rf.feature_importances_
feature_labels = ElementFraction().feature_labels()
features_and_their_importances = dict(zip(feature_labels, importances))
plot_importances(features_and_their_importances)

# How does this compare to published models for formation energy?
- Here is the state of the art for a similar prediction task using only chemical composition (from [this paper](https://www.nature.com/articles/s41467-020-19964-7))
![](../files/roost.png)



## What if we also know the crystal structure?
- from [this paper](https://www.nature.com/articles/s41586-023-06735-9)

![](../files/gnome.png)

# Let's look at a crystal structure file

In [ ]:
random_index = 3935

structure = df['structure'].values[random_index]

print(structure)

# Featurizing crystal structures introduces new challenges:
- how do we create "fixed-length" vectors to feed to ML models?
- how do we handle "invariances" (eg wrt to supercell expansions)?
- how do we encode chemical identities in addition to atomic positions?

# Let's explore one approach to featurizing structure
- the radial distribution function
- here, our features will simply be a binned histogram of distances (r) and the values for each feature will be g(r)

## This will nicely handle "fixed-length" problem and supercell invariances

In [ ]:
rdf = RadialDistributionFunction()
unitcell = df.structure.get(df.formula == 'AlN').values[0]
supercell = unitcell.make_supercell(2,2,2)

unit_rdf = rdf.featurize(unitcell)
super_rdf = rdf.featurize(supercell)

fig = plt.figure()
ax = plt.subplot(211)
ax = plt.bar(list(range(len(unit_rdf))), unit_rdf)
ax = plt.subplot(212)
ax = plt.bar(list(range(len(super_rdf))), super_rdf)

## Let's compare our three "fingerprints"

In [ ]:
formula = 'NaMnSe2'
elfrac_vector = df['elfrac_vector'].get(df.formula == formula).values[0]
matminer_vector = df['matminer_vector'].get(df.formula == formula).values[0]
rdf_vector = rdf.featurize(df['structure'].get(df.formula == formula).values[0])

fig = plt.figure()
ax1 = plt.subplot(311)
ax1 = plt.bar(list(range(len(elfrac_vector))), elfrac_vector)
ax2 = plt.subplot(312)
ax2 = plt.bar(list(range(len(matminer_vector))), matminer_vector)
ax3 = plt.subplot(313)
ax3 = plt.bar(list(range(len(rdf_vector))), rdf_vector)


## Let's apply the rdf vector to all materials (this may take a minute)

In [ ]:
structures = df.structure.values
rdfs = [rdf.featurize(s) for s in structures]
df['rdf_vector'] = rdfs

# A common way to understand representations is through "similarities"
- cosine similarity --> popular for high-dimensional representations (vectors)
- Euclidean similarity --> more amenable to low-dimensional representations (common "distance")

In [ ]:
def similarity(a, b, method='cosine'):
    """
    Args:
        a (np.array) : a feature vector for a material
        b (np.array) : a feature vector for another material
        method (str) : "cosine" or "euclidean"

    Returns:
        if method == 'cosine':
            the cosine similarity (1 - the cosine distance between the two vectors) (float)
        if method == 'euclidean':
            the inverse Euclidean distance (float)
    """    
    a = np.nan_to_num(a)
    b = np.nan_to_num(b)
    if method == 'cosine':
        return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    elif method == 'euclidean':
        dist = np.linalg.norm(a-b)
        if dist != 0:
            return 1/dist
        else:
            return 1e6
    else:
        raise NotImplementedError

## Group activities
**Guidelines**
- Find the 5 materials in your dataset that are most similar to aluminum nitride (AlN).
- Consider the choice of representation. Does it matter?

In [ ]:
###### STUDENTS CODE HERE

## Let's fit some models using the rdf vector

In [ ]:
scores, rf = CV('rdf_vector')


In [ ]:
importances = rf.feature_importances_
feature_labels = list(range(len(importances)))
features_and_their_importances = dict(zip(feature_labels, importances))
plot_importances(features_and_their_importances, n_features_to_plot=10)

# More processing (and probably data!) required.. and algorithm matters! 
- Kernel ridge regression is popular with these fingerprint-like features
  - read more about KRR [here](https://dmol.pub/ml/kernel.html) and [here](https://github.com/fullmetalfelix/ML-CSC-tutorial/blob/master/krr_homo.ipynb) (among many other places)
- also more approaches that are especially amenable to kernel-based methods (e.g., see [SOAP](https://singroup.github.io/dscribe/0.3.x/tutorials/soap.html)) and many more that are amenable to deep learning (e.g., [equivariant neural networks](https://github.com/mir-group/nequip))